In [15]:
import pandas as pd 
# from imutils.video import WebcamVideoStream
# import imutils
import matplotlib.pyplot as plt
import cv2
import numpy as np
import threading
import mediapipe as mp
from matplotlib import pyplot as plt
import time
import os

# from imutils.video import FPS
mp.solutions.drawing_utils._VISIBILITY_THRESHOLD = 0.01
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

landmark_lb = [ "Elbow(L)", "Elbow(R)", "Shoulder(L)", "Shoulder(R)",  "Hip(L)", "Hip(R)", "Knee(L)", "Knee(R)", "FOOT(L)", "FOOT(R)", "NECK(H)", "NECK(V)"]

z_scale_factor = np.array([ 1, 
                           1, 1, 1, 1, 1, 
                           1, 1, 1, 1, 1, 
                           0.2, 0.2, 0.2, 0.2, 0.2, 
                           0.2, 1, 1, 1, 1, 
                           1, 1, 0.5, 0.5, 0.5, 
                           0.5, 0.5, 0.5, 1, 1, 
                           0.5, 0.5])

def get_angle(a, b, c):
    ba = a - b
    bc = c - b
    
    ba[2] = 0
    bc[2] = 0

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

def get_stand(a, b):
    ba = a - b
    y_differ = ba[1]
    z_differ = ba[2]
    return [y_differ,z_differ]

def get_gradient(a,b):
    c = [b[0],a[1],a[2]]
    
    ba = b - a
    ac = c - a
  
    ba[2] = 0
    ac[2] = 0
    #print('ba',ba,'\nac',ac)
    cosine_angle = np.dot(ba, ac) / (np.linalg.norm(ba) * np.linalg.norm(ac))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)
    

def get_joint_angle(data):
    #angle 입력하는 곳
    joints = [[11,23,24],[12,24,23],[15,13,11],[16,14,12],[11,0,12]
]
  
    stand = [[23,24],[7,8]]
    
    return np.array([ get_angle( *data[j, :3]) if len(j) == 3 else get_gradient( *data[j, :3]) for j in joints]),np.array([ get_stand( *data[j, :3]) for j in stand])


def angle_differ(arr):
    angle_arr = []
    for i in range(len(arr)-1):
        appends = np.abs(arr[i+1] - arr[i])
        appends = appends.tolist()
        angle_arr.append(appends)
    angle_arr = np.array(angle_arr)
    return angle_arr
#threshold csv 파일 값 읽어오기
    

# 디스플레이 부분


# Display function
# ================
def view():
    #사진 넣을 경우
    #cap = cv2.imread('./pic/12.jpg', cv2.IMREAD_COLOR)
    #비디오 넣을 경우
    #cap = cv2.VideoCapture('./33.mp4')
    
    lo = '../48/'
    data = os.listdir(lo)
    print(data)
    
    coords = np.zeros((1, 38,4))

    cnt = 0
    cur_id = 0
    create = None
    
    #i = 0
    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False) as pose:
    
        
        for img in data:
            #비디오
            #_,frame = cap.read()
            #사진
            cap = cv2.imread(lo+img, cv2.IMREAD_COLOR)
            frame = cap

            image_height, image_width, _ = frame.shape
            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            if results.pose_landmarks is None:
                print('nope')
                continue


            cur_coord = np.array([(results.pose_landmarks.landmark[i].x, 
                              results.pose_landmarks.landmark[i].y, 
                              results.pose_landmarks.landmark[i].z, 
                              results.pose_landmarks.landmark[i].visibility) for i in range(33)])#if results.pose_landmarks.landmark[i].visibility > 0.2 else (0,0,0,0) for i in range(33) ])


            #cur_coord[:,2] = cur_coord[:,2] *0


            #cur_coord[:, 2] = cur_coord[:, 2] * z_scale_factor

            cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[31:32, :2], cur_coord[27:28, 2:], axis=0), (-1,4)), axis= 0)
            cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[32:33, :2], cur_coord[28:29, 2:], axis=0), (-1,4)), axis= 0)
            cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[7:8, :2], cur_coord[8:9, 2:], axis=0), (-1,4)), axis= 0)
            cur_coord = np.append( cur_coord, np.reshape([ cur_coord[8, 0], cur_coord[7, 1], cur_coord[8, 2], cur_coord[8, 3]], (-1,4)), axis= 0)
            cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[0:1, :2], cur_coord[8:9, 2:], axis=0), (-1,4)), axis= 0)


            coords = np.append(coords[-2:], np.reshape(cur_coord, (-1,38,4)), axis=0)

            coord = np.median(coords, axis =0 )

            input_angles, input_stands = get_joint_angle(coord) 
            
            #print('입력한 조인트 각도',input_angles)
            #print('잘 서있는지 확인 하는 각도',input_stands)

            #이미지에 텍스트 표기하는 부분
            text = "1_:{:.0f} , 2_:{:.0f},3_:{:.0f} , 4_:{:.0f}, 5_:{:.0f}".format(input_angles[0],input_angles[1],input_angles[2],input_angles[3],input_angles[4]) #,input_stands[0][0])#,input_stands[1][0])
            print(img,text,'\n')
            cv2.putText(frame, text, (10, 40), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1, (255, 255, 255), 1)
            
            #cv2.imshow('frame', frame)
            cv2.imwrite(str(img),frame)

#             if create is None:
#                 fourcc = cv2.VideoWriter_fourcc(*'XVID')
#                 create = cv2.VideoWriter("output.avi", fourcc, 30, (640, 480), True)

#             create.write(frame)

            key = cv2.waitKey(1) & 0xFF

            if key == ord("q"):
                break
                
            i=+1
            time.sleep(1)
                
# ================
view()
cv2.destroyAllWindows()

['48_3.png', '48_2.png', '48_0.png', '48_1.png', '48_5.png', '48_4.png', '48_6.png', '48_7.png', '48_8.png']
48_3.png 1_:104 , 2_:100,3_:172 , 4_:26, 5_:104 

48_2.png 1_:102 , 2_:101,3_:175 , 4_:26, 5_:104 

48_0.png 1_:109 , 2_:99,3_:173 , 4_:12, 5_:114 

48_1.png 1_:111 , 2_:97,3_:174 , 4_:16, 5_:114 

48_5.png 1_:106 , 2_:101,3_:177 , 4_:136, 5_:97 

48_4.png 1_:105 , 2_:102,3_:133 , 4_:154, 5_:100 

48_6.png 1_:104 , 2_:103,3_:3 , 4_:157, 5_:101 

48_7.png 1_:79 , 2_:119,3_:10 , 4_:177, 5_:37 

48_8.png 1_:76 , 2_:121,3_:10 , 4_:176, 5_:37 

